## **Описание проекта**
**SQL**

Наша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Наша первая задача — проанализировать базу данных.
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

**Цель:** ответить на вопросы поставленные заказчиком, сформулировать ценностное предложение для нового продукта

**Описание данных:**

Таблица books
Содержит данные о книгах:

    book_id — идентификатор книги;
    author_id — идентификатор автора;
    title — название книги;
    num_pages — количество страниц;
    publication_date — дата публикации книги;
    publisher_id — идентификатор издателя.

Таблица authors
Содержит данные об авторах:

    author_id — идентификатор автора;
    author — имя автора.

Таблица publishers
Содержит данные об издательствах:

    publisher_id — идентификатор издательства;
    publisher — название издательства;

Таблица ratings
Содержит данные о пользовательских оценках книг:

    rating_id — идентификатор оценки;
    book_id — идентификатор книги;
    username — имя пользователя, оставившего оценку;
    rating — оценка книги.

Таблица reviews
Содержит данные о пользовательских обзорах:

    review_id — идентификатор обзора;
    book_id — идентификатор книги;
    username — имя автора обзора;
    text — текст обзора.
`


**План проекта:**

Ответим на вопросы: 

[Посчитайте, сколько книг вышло после 1 января 2000 года;](#first)
  
[Для каждой книги посчитайте количество обзоров и среднюю оценку;](#second)
  
[Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;](#third)

[Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;](#fourth)

[Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.](#fifth)

[Общий вывод](#conclution)

## **Подключимся к базе данных**

In [13]:
# импортируем библиотеки 
import pandas as pd 
from sqlalchemy import create_engine 
# устанавливаем параметры 
db_config = {'user': 'praktikum_student', # имя пользователя 
                     'pwd': 'Sdf4$2;d-d30pp', # пароль 
                     'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net', 
                     'port': 6432, # порт подключения 
                     'db': 'data-analyst-final-project-db'} # название базы данных 
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'], 
                                                db_config['pwd'], 
                                                db_config['host'], 
                                                db_config['port'], 
                                                db_config['db']) 
# сохраняем коннектор 
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 


# Формируем sql-запрос.
query = ''' SELECT *
            FROM books
        '''

# Выполняем запрос и сохраняем результат
# выполнения в DataFrame.
# Sqlalchemy автоматически установит названия колонок
# такими же, как у таблицы в БД. 
books = pd.io.sql.read_sql(query, con = engine) 

print(books.info())
books.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
book_id             1000 non-null int64
author_id           1000 non-null int64
title               1000 non-null object
num_pages           1000 non-null int64
publication_date    1000 non-null object
publisher_id        1000 non-null int64
dtypes: int64(4), object(2)
memory usage: 47.0+ KB
None


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [10]:
query = ''' SELECT *
            FROM publishers
        '''
publishers = pd.io.sql.read_sql(query, con = engine) 

print(publishers.info())
publishers.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 2 columns):
publisher_id    340 non-null int64
publisher       340 non-null object
dtypes: int64(1), object(1)
memory usage: 5.4+ KB
None


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [11]:
query = ''' SELECT *
            FROM ratings
        '''
ratings = pd.io.sql.read_sql(query, con = engine) 

print(ratings.info())
ratings.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6456 entries, 0 to 6455
Data columns (total 4 columns):
rating_id    6456 non-null int64
book_id      6456 non-null int64
username     6456 non-null object
rating       6456 non-null int64
dtypes: int64(3), object(1)
memory usage: 201.9+ KB
None


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [12]:
query = ''' SELECT *
            FROM reviews
        '''
reviews = pd.io.sql.read_sql(query, con = engine) 

print(reviews.info())
reviews.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2793 entries, 0 to 2792
Data columns (total 4 columns):
review_id    2793 non-null int64
book_id      2793 non-null int64
username     2793 non-null object
text         2793 non-null object
dtypes: int64(2), object(2)
memory usage: 87.4+ KB
None


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## **Ответим на поставленные в проекте вопросы**

<a id='first'></a>  
1. Посчитаем, сколько книг вышло после 1 января 2000 года;

In [5]:
query = ''' SELECT COUNT(book_id)
            FROM books
            WHERE publication_date >= '2000-01-01'
        '''
count_books = pd.io.sql.read_sql(query, con = engine)
print('Количество книг вышло после 1 января 2000 года:',count_books['count'][0])

Количество книг вышло после 1 января 2000 года: 821


<a id='second'></a>  
2. Для каждой книги посчитайте количество обзоров и среднюю оценку;

In [79]:
query = ''' SELECT books_rating.book_id,
                   title,
                   review_qty,
                   avr_rating
            FROM (SELECT b.book_id,
                         title,
                  ROUND (AVG(rating),2) as avr_rating
                  FROM books b
                      join ratings r 
                          on b.book_id = r.book_id
                  GROUP BY b.book_id) as books_rating
            LEFT JOIN(SELECT b.book_id,
                             COUNT(review_id) as review_qty
                      FROM books b
                          left join reviews rw 
                              on b.book_id = rw.book_id
                      GROUP BY b.book_id
            ) AS books_review ON books_rating.book_id = books_review.book_id
            ORDER BY review_qty desc
       
        '''
books_info= pd.io.sql.read_sql(query, con = engine)
books_info

,book_id,title,review_qty,avr_rating
0,948,Twilight (Twilight #1),7,3.66
1,695,The Curious Incident of the Dog in the Night-Time,6,4.08
2,299,Harry Potter and the Chamber of Secrets (Harry...,6,4.29
3,696,The Da Vinci Code (Robert Langdon #2),6,3.83
4,734,The Glass Castle,6,4.21
...,...,...,...,...
995,672,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.00
996,808,The Natural Way to Draw,0,3.00
997,221,Essential Tales and Poems,0,4.00
998,387,Leonardo's Notebooks,0,4.00


Всего в базе 1000 книг, самое большое количество обзоров у Twilight (Twilight #1), с оценкой 3,66

<a id='third'></a>  
3. Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;

In [35]:
query = ''' SELECT publisher,
            COUNT(title) as books_qty
            FROM books b
                join publishers p 
                    on b.publisher_id = p.publisher_id
            WHERE num_pages > 50
            GROUP BY publisher
            ORDER BY books_qty desc
            LIMIT 1
        '''
publisher_info= pd.io.sql.read_sql(query, con = engine)
print('Издательство, которое выпустило наибольшее число книг толще 50 страниц:')
publisher_info

Издательство, которое выпустило наибольшее число книг толще 50 страниц:


,publisher,books_qty
0,Penguin Books,42


<a id='fourth'></a>  
4. Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;

In [83]:
query = ''' SELECT author,
            ROUND(AVG(rating),2) as avg_rating
            FROM books b
                join ratings r 
                    on b.book_id = r.book_id
                join authors a
                    on b.author_id = a.author_id   
            WHERE b.book_id in (SELECT b.book_id
                            FROM books b
                                join ratings r 
                                    on b.book_id = r.book_id               
                            GROUP BY b.book_id
                            HAVING COUNT(rating) >= 50)
            GROUP BY author
            ORDER BY avg_rating desc
            LIMIT 1
        '''
books_rating= pd.io.sql.read_sql(query, con = engine)
print('Автор с самой высокой средней оценкой книг:')
books_rating

Автор с самой высокой средней оценкой книг:


,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.29


<a id='fifth'></a>  
5.Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [9]:
query = ''' SELECT 
            ROUND(AVG(SUBQ.review_qty),2) as avg_review_qty
            FROM (SELECT username,
                  COUNT(review_id) as review_qty
                  FROM books b
                      join reviews rw 
                          on b.book_id = rw.book_id
                  GROUP BY username) as SUBQ
            WHERE username in (SELECT username
                               FROM books b
                                   join ratings r
                                       on b.book_id = r.book_id
                               GROUP BY username
                               HAVING COUNT(rating) > 50)
        '''
user_rating= pd.io.sql.read_sql(query, con = engine)
print('Среднее количество обзоров от пользователей, которые поставили больше 50 оценок:',user_rating['avg_review_qty'][0])

Среднее количество обзоров от пользователей, которые поставили больше 50 оценок: 24.33


<a id='conclution'></a>  
## Общий вывод

    Анализ проведен, на поставленные вопросы даны ответы:

    Всего в базе 1000 кник,из них 821 книги вышло после 1 января 2000 года. Наибольшее количество обзоров собрала книга Twilight (Twilight #1), со средней оценкой 3.66. Вторая в рейтинге книга The Curious Incident of the Dog in the Night-Time, у нее оценка намного выше -4.08.
    Издательство, которое выпустило наибольшее число книг толще 50 страниц - Penguin Books
    Автор с самой высокой средней оценкой книг: J.K. Rowling/Mary GrandPré
    Среднее количество обзоров от пользователей, которые поставили больше 50 оценок: 24.33